## 뉴스 크롤링

In [7]:
import requests
from bs4 import BeautifulSoup
import time

### 사회

In [26]:
def ex_tag(sid, page):
    ### 뉴스 분야(sid)와 페이지(page)를 입력하면 그에 대한 링크들을 리스트로 추출하는 함수 ###
    
    ## 1.
    url = f"https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1={sid}"\
    "#&date=%2000:00:00&page={page}"
    html = requests.get(url, headers={"User-Agent": "Mozilla/5.0"\
    "(Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) "\
    "Chrome/110.0.0.0 Safari/537.36"})
    soup = BeautifulSoup(html.text, "lxml")
    a_tag = soup.find_all("a")
    
    ## 2.
    tag_lst = []
    for a in a_tag:
        if "href" in a.attrs:  # href가 있는것만 고르는 것
            if (f"sid={sid}" in a["href"]) and ("article" in a["href"]):
                tag_lst.append(a["href"])
                
    return tag_lst

# 경제는 101, 사회는 102, IT/과학은 105

In [36]:
def re_tag(sid):
    re_lst = []
    for i in range(10): # 오늘 기준으로 페이지 확인 
        lst = ex_tag(sid, i+1)
        re_lst.extend(lst)

    # 중복 제거
    re_set = set(re_lst)
    re_lst = list(re_set)
    
    return re_lst

In [50]:
all_hrefs = {}
sids = [102]  # 분야 리스트 - 경제는 101, 사회는 102, IT/과학은 105

# 각 분야별로 링크 수집해서 딕셔너리에 저장
for sid in sids:
    sid_data = re_tag(sid)
    all_hrefs[sid] = sid_data

In [46]:
def art_crawl(all_hrefs, sid, index):
    """
    sid와 링크 인덱스를 넣으면 기사제목, 날짜, 본문을 크롤링하여 딕셔너리를 출력하는 함수 
    
    Args: 
        all_hrefs(dict): 각 분야별로 100페이지까지 링크를 수집한 딕셔너리 (key: 분야(sid), value: 링크)
        sid(int): 분야 [101: 경제, 102: 사회, 105: IT/과학]
        index(int): 링크의 인덱스
    
    Returns:
        dict: 기사제목, 날짜, 본문이 크롤링된 딕셔너리
    
    """
    art_dic = {}
    
    ## 1.
    title_selector = "#title_area > span"
    date_selector = "#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans"\
    "> div.media_end_head_info_datestamp > div:nth-child(1) > span"
    main_selector = "#dic_area"
    
    url = all_hrefs[sid][index]
    html = requests.get(url, headers = {"User-Agent": "Mozilla/5.0 "\
    "(Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)"\
    "Chrome/110.0.0.0 Safari/537.36"})
    soup = BeautifulSoup(html.text, "lxml")
    
    ## 2.
    # 제목 수집
    title = soup.select(title_selector)
    title_lst = [t.text for t in title]
    title_str = "".join(title_lst)
    
    # 날짜 수집
    date = soup.select(date_selector)
    date_lst = [d.text for d in date]
    date_str = "".join(date_lst)
    
    # 본문 수집
    main = soup.select(main_selector)
    main_lst = []
    for m in main:
        m_text = m.text
        m_text = m_text.strip()
        main_lst.append(m_text)
    main_str = "".join(main_lst)
    
    ## 3.
    art_dic["title"] = title_str
    art_dic["date"] = date_str
    art_dic["main"] = main_str
    
    return art_dic

In [51]:
# 모든 섹션의 데이터 수집 (제목, 날짜, 본문, section, url)
section_lst = [102]
section = section_lst[0]
artdic_lst = []

#for section in tqdm(section_lst):
for i in range(len(all_hrefs[section])):
    art_dic = art_crawl(all_hrefs, section, i)
    art_dic["section"] = section
    art_dic["url"] = all_hrefs[section][i]
    artdic_lst.append(art_dic)

In [ ]:
import pandas as pd

art_df_so = pd.DataFrame(artdic_lst)
art_df_so

In [53]:
# 해당 크롤링에 맞게 크롤링 저장
art_df_so.to_csv("society.csv", encoding="utf-8-sig")

### 경제

In [ ]:
def ex_tag(sid, page):
    ### 뉴스 분야(sid)와 페이지(page)를 입력하면 그에 대한 링크들을 리스트로 추출하는 함수 ###
    
    ## 1.
    url = f"https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1={sid}"\
    "#&date=%2000:00:00&page={page}"
    html = requests.get(url, headers={"User-Agent": "Mozilla/5.0"\
    "(Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) "\
    "Chrome/110.0.0.0 Safari/537.36"})
    soup = BeautifulSoup(html.text, "lxml")
    a_tag = soup.find_all("a")
    
    ## 2.
    tag_lst = []
    for a in a_tag:
        if "href" in a.attrs:  # href가 있는것만 고르는 것
            if (f"sid={sid}" in a["href"]) and ("article" in a["href"]):
                tag_lst.append(a["href"])
                
    return tag_lst

# 경제는 101, 사회는 102, IT/과학은 105

In [ ]:
def re_tag(sid):
    re_lst = []
    for i in range(10): # 오늘 기준으로 페이지 확인 
        lst = ex_tag(sid, i+1)
        re_lst.extend(lst)

    # 중복 제거
    re_set = set(re_lst)
    re_lst = list(re_set)
    
    return re_lst

In [ ]:
all_hrefs = {}
sids = [101]  # 분야 리스트 - 경제는 101, 사회는 102, IT/과학은 105

# 각 분야별로 링크 수집해서 딕셔너리에 저장
for sid in sids:
    sid_data = re_tag(sid)
    all_hrefs[sid] = sid_data

In [ ]:
def art_crawl(all_hrefs, sid, index):
    """
    sid와 링크 인덱스를 넣으면 기사제목, 날짜, 본문을 크롤링하여 딕셔너리를 출력하는 함수 
    
    Args: 
        all_hrefs(dict): 각 분야별로 100페이지까지 링크를 수집한 딕셔너리 (key: 분야(sid), value: 링크)
        sid(int): 분야 [101: 경제, 102: 사회, 105: IT/과학]
        index(int): 링크의 인덱스
    
    Returns:
        dict: 기사제목, 날짜, 본문이 크롤링된 딕셔너리
    
    """
    art_dic = {}
    
    ## 1.
    title_selector = "#title_area > span"
    date_selector = "#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans"\
    "> div.media_end_head_info_datestamp > div:nth-child(1) > span"
    main_selector = "#dic_area"
    
    url = all_hrefs[sid][index]
    html = requests.get(url, headers = {"User-Agent": "Mozilla/5.0 "\
    "(Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)"\
    "Chrome/110.0.0.0 Safari/537.36"})
    soup = BeautifulSoup(html.text, "lxml")
    
    ## 2.
    # 제목 수집
    title = soup.select(title_selector)
    title_lst = [t.text for t in title]
    title_str = "".join(title_lst)
    
    # 날짜 수집
    date = soup.select(date_selector)
    date_lst = [d.text for d in date]
    date_str = "".join(date_lst)
    
    # 본문 수집
    main = soup.select(main_selector)
    main_lst = []
    for m in main:
        m_text = m.text
        m_text = m_text.strip()
        main_lst.append(m_text)
    main_str = "".join(main_lst)
    
    ## 3.
    art_dic["title"] = title_str
    art_dic["date"] = date_str
    art_dic["main"] = main_str
    
    return art_dic

In [ ]:
# 모든 섹션의 데이터 수집 (제목, 날짜, 본문, section, url)
section_lst = [101]
section = section_lst[0]
artdic_lst = []

#for section in tqdm(section_lst):
for i in range(len(all_hrefs[section])):
    art_dic = art_crawl(all_hrefs, section, i)
    art_dic["section"] = section
    art_dic["url"] = all_hrefs[section][i]
    artdic_lst.append(art_dic)

In [ ]:
import pandas as pd

art_df_eco = pd.DataFrame(artdic_lst)
art_df_eco

In [ ]:
# 해당 크롤링에 맞게 크롤링 저장
art_df_eco.to_csv("economy.csv", encoding="utf-8-sig")

### IT

In [ ]:
def ex_tag(sid, page):
    ### 뉴스 분야(sid)와 페이지(page)를 입력하면 그에 대한 링크들을 리스트로 추출하는 함수 ###
    
    ## 1.
    url = f"https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1={sid}"\
    "#&date=%2000:00:00&page={page}"
    html = requests.get(url, headers={"User-Agent": "Mozilla/5.0"\
    "(Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) "\
    "Chrome/110.0.0.0 Safari/537.36"})
    soup = BeautifulSoup(html.text, "lxml")
    a_tag = soup.find_all("a")
    
    ## 2.
    tag_lst = []
    for a in a_tag:
        if "href" in a.attrs:  # href가 있는것만 고르는 것
            if (f"sid={sid}" in a["href"]) and ("article" in a["href"]):
                tag_lst.append(a["href"])
                
    return tag_lst

# 경제는 101, 사회는 102, IT/과학은 105

In [ ]:
def re_tag(sid):
    re_lst = []
    for i in range(10): # 오늘 기준으로 페이지 확인 
        lst = ex_tag(sid, i+1)
        re_lst.extend(lst)

    # 중복 제거
    re_set = set(re_lst)
    re_lst = list(re_set)
    
    return re_lst

In [ ]:
all_hrefs = {}
sids = [105]  # 분야 리스트 - 경제는 101, 사회는 102, IT/과학은 105

# 각 분야별로 링크 수집해서 딕셔너리에 저장
for sid in sids:
    sid_data = re_tag(sid)
    all_hrefs[sid] = sid_data

In [ ]:
def art_crawl(all_hrefs, sid, index):
    """
    sid와 링크 인덱스를 넣으면 기사제목, 날짜, 본문을 크롤링하여 딕셔너리를 출력하는 함수 
    
    Args: 
        all_hrefs(dict): 각 분야별로 100페이지까지 링크를 수집한 딕셔너리 (key: 분야(sid), value: 링크)
        sid(int): 분야 [101: 경제, 102: 사회, 105: IT/과학]
        index(int): 링크의 인덱스
    
    Returns:
        dict: 기사제목, 날짜, 본문이 크롤링된 딕셔너리
    
    """
    art_dic = {}
    
    ## 1.
    title_selector = "#title_area > span"
    date_selector = "#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans"\
    "> div.media_end_head_info_datestamp > div:nth-child(1) > span"
    main_selector = "#dic_area"
    
    url = all_hrefs[sid][index]
    html = requests.get(url, headers = {"User-Agent": "Mozilla/5.0 "\
    "(Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)"\
    "Chrome/110.0.0.0 Safari/537.36"})
    soup = BeautifulSoup(html.text, "lxml")
    
    ## 2.
    # 제목 수집
    title = soup.select(title_selector)
    title_lst = [t.text for t in title]
    title_str = "".join(title_lst)
    
    # 날짜 수집
    date = soup.select(date_selector)
    date_lst = [d.text for d in date]
    date_str = "".join(date_lst)
    
    # 본문 수집
    main = soup.select(main_selector)
    main_lst = []
    for m in main:
        m_text = m.text
        m_text = m_text.strip()
        main_lst.append(m_text)
    main_str = "".join(main_lst)
    
    ## 3.
    art_dic["title"] = title_str
    art_dic["date"] = date_str
    art_dic["main"] = main_str
    
    return art_dic

In [ ]:
# 모든 섹션의 데이터 수집 (제목, 날짜, 본문, section, url)
section_lst = [105]
section = section_lst[0]
artdic_lst = []

#for section in tqdm(section_lst):
for i in range(len(all_hrefs[section])):
    art_dic = art_crawl(all_hrefs, section, i)
    art_dic["section"] = section
    art_dic["url"] = all_hrefs[section][i]
    artdic_lst.append(art_dic)

In [ ]:
import pandas as pd

art_df_it = pd.DataFrame(artdic_lst)
art_df_it

In [ ]:
# 해당 크롤링에 맞게 크롤링 저장
art_df_it.to_csv("IT.csv", encoding="utf-8-sig")

### 스포츠

In [ ]:
#스포츠 제목
from bs4 import BeautifulSoup as bs
url = 'https://sports.news.naver.com/ranking/index.nhn?date=20230602'

#step3.requests 패키지의 함수를 이용해 url의 html 문서를 가져온다.
response = requests.get(url)
html_text=response.text

#step4.bs4 패키지의 함수를 이용해서 html 문서를 파싱한다.
soup = bs(html_text, 'html.parser')

#step6.bs4 패키지의 select 함수와 선택자 개념을 이용해서 뉴스기사 제목을 모두 가져온다.
titles = soup.select('a.title')

n=0
for i in titles:
    title = i.get_text()
    print(title.strip())
    n= n + 1
    if(n==10):
        break

In [ ]:
#스포츠 뉴스
from bs4 import BeautifulSoup as bs
url = 'https://sports.news.naver.com/ranking/index'

#step3.requests 패키지의 함수를 이용해 url의 html 문서를 가져온다.
response = requests.get(url)
html_text=response.text

#step4.bs4 패키지의 함수를 이용해서 html 문서를 파싱한다.
soup = bs(html_text, 'html.parser')

#step6.bs4 패키지의 select 함수와 선택자 개념을 이용해서 뉴스기사 제목을 모두 가져온다.
titles = soup.select('a.title')

n=0
news ={}
for i in titles:
    url = 'https://sports.news.naver.com/'+i.attrs["href"]
    response = requests.get(url, headers={'User-agent': 'Mozila/5.0'})      # to avoid error use headers
    html = response.text                                                    # for each url get html
    soup = BeautifulSoup(html, "html.parser")                               # for each html make soup
    headline = soup.select_one("div.news_headline")
    title = headline.select_one("h4").text.strip()
    #print(headline.select_one("h4").text)
    content = soup.select_one("#newsEndContents")                                  # get the body
#     print(content.text)
    news[title] = (content.text.strip())
    time.sleep(0.3)
    n= n + 1
    if(n==10):
        break
        
(news)

In [ ]:
import pandas as pd

#데이터 프레임 만들기
news_df = pd.DataFrame({'title':news.keys(),'content':news.values()})
news_df

In [ ]:
news_df.to_csv('sports.csv' ,encoding='utf-8-sig',index=False)

### 연예

In [ ]:
#연예 제목
from bs4 import BeautifulSoup as bs
url = 'https://entertain.naver.com/ranking'

#step3.requests 패키지의 함수를 이용해 url의 html 문서를 가져온다.
response = requests.get(url)
html_text=response.text

#step4.bs4 패키지의 함수를 이용해서 html 문서를 파싱한다.
soup = bs(html_text, 'html.parser')

#step6.bs4 패키지의 select 함수와 선택자 개념을 이용해서 뉴스기사 제목을 모두 가져온다.
titles = soup.select('#ranking_news > div > div.rank_lst')

n=0
for i in titles:
    title = i.get_text()
    print(title.strip())
#     n= n + 1
#     if(n==10):
#         break
# titles

In [ ]:
#연예 뉴스
from bs4 import BeautifulSoup as bs
url = 'https://entertain.naver.com/ranking'

#step3.requests 패키지의 함수를 이용해 url의 html 문서를 가져온다.
response = requests.get(url)
html_text=response.text

#step4.bs4 패키지의 함수를 이용해서 html 문서를 파싱한다.
soup = bs(html_text, 'html.parser')

#step6.bs4 패키지의 select 함수와 선택자 개념을 이용해서 뉴스기사 제목을 모두 가져온다.
titles = soup.select('#ranking_news > div > div.rank_lst')
newsList = (titles[0].select('a'))
#n=0
news ={}
for i in newsList:
#     print(i)
    url = 'https://entertain.naver.com/'+i.attrs["href"]
    response = requests.get(url, headers={'User-agent': 'Mozila/5.0'})      # to avoid error use headers
    html = response.text                                                    # for each url get html
    soup = BeautifulSoup(html, "html.parser")                               # for each html make soup
    headline = soup.select_one("#content > div.end_ct > div > h2")
    title = headline.text.strip()
    #print(headline.text)
    content = soup.select_one("#articeBody")                                  # get the body
    #print(content.text)
    news[title] = (content.text.strip())
    time.sleep(0.3)
    #n= n + 1
    #if(n==10):
    #    break
        
(news)

In [ ]:
import pandas as pd

#데이터 프레임 만들기
news_df = pd.DataFrame({'title':news.keys(),'content':news.values()})
(news_df)

In [ ]:
news_df.to_csv('entertain.csv' ,encoding='utf-8-sig',index=False)